In [2]:
import numpy as np
#import pandas as pd

import tensorflow.keras as keras
from tensorflow.keras import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from PIL import Image
from sklearn.decomposition import PCA

from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

from ultralytics import YOLO

2024-04-18 16:19:28.586178: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [1]:
train = keras.utils.image_dataset_from_directory(
    directory='../../data/Faces updated',
    labels='inferred',
    label_mode='categorical',
    batch_size=140,
    image_size=(1000, 1000),
    validation_split = 0.2,
    subset = 'training',
    seed = 50,
   
    color_mode = 'grayscale'
    )

validation = keras.utils.image_dataset_from_directory(
    directory='../../data/Faces updated',
    labels='inferred',
    label_mode='categorical',
    batch_size=140,
    image_size=(1000, 1000),
    validation_split = 0.2,
    subset = 'validation',
    seed = 50,
   # class_names = ['Happy', 'Sad', 'Neutral'],
    color_mode = 'grayscale'
    )

NameError: name 'keras' is not defined

In [43]:
#convert to numpy images and then perform PCA to extract the values
#convert images to numpy
train_processed_numpy = train.as_numpy_iterator()
validation_processed_numpy = validation.as_numpy_iterator()

X_train, y_train = train_processed_numpy.next()
X_val, y_val = validation_processed_numpy.next()

In [44]:
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1] * X_train.shape[2]))
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1] * X_val.shape[2]))

In [45]:
pca = PCA(n_components = 50)

X_train_lower_dim = pca.fit_transform(X_train)

In [46]:
pca.explained_variance_ratio_

#As can be seen from the output below, the first 50 components explain a substantial proportsion of the variance
#include train accuracy as well

array([     0.2146,     0.14558,     0.09987,    0.059061,    0.054691,    0.052791,    0.029249,    0.026064,    0.022926,    0.017457,    0.015267,    0.014115,    0.012153,    0.010603,    0.010218,   0.0097132,   0.0089257,   0.0083896,   0.0081183,   0.0076957,   0.0073769,   0.0067631,   0.0063553,   0.0062902,
         0.0062499,   0.0058621,   0.0053985,   0.0051481,   0.0048855,   0.0047491,   0.0043784,   0.0041223,   0.0038568,   0.0037828,    0.003607,   0.0035339,   0.0032696,   0.0031205,   0.0030526,   0.0028445,   0.0028107,   0.0027358,   0.0026297,   0.0026019,   0.0024725,   0.0023883,   0.0022195,   0.0021563,
         0.0021374,   0.0020926], dtype=float32)

In [47]:
#fitting an SVC object on this

svm = SVC()

svm.fit(X_train_lower_dim, np.argmax(y_train, axis = 1))

SVC()

In [48]:
#transforming the test data into the lower dimension
X_val_lower_dim = pca.transform(X_val)
y_pred = svm.predict(X_val_lower_dim)

In [49]:
accuracy_score(np.argmax(y_val, axis = 1),y_pred)

0.43333333333333335

# After face cropping

In [50]:
train = keras.utils.image_dataset_from_directory(
    directory='../../data/Faces updated',
    labels='inferred',
    label_mode='categorical',
    batch_size=140,
    image_size=(1000, 1000),
    validation_split = 0.2,
    subset = 'training',
    seed = 50,
   )

validation = keras.utils.image_dataset_from_directory(
    directory='../../data/Faces updated',
    labels='inferred',
    label_mode='categorical',
    batch_size=140,
    image_size=(1000, 1000),
    validation_split = 0.2,
    subset = 'validation',
    seed = 50,
  
    )

Found 154 files belonging to 3 classes.
Using 124 files for training.
Found 154 files belonging to 3 classes.
Using 30 files for validation.


In [51]:
#convert to numpy images and then perform PCA to extract the values
#convert images to numpy
train_processed_numpy = train.as_numpy_iterator()
validation_processed_numpy = validation.as_numpy_iterator()

X_train, y_train = train_processed_numpy.next()
X_val, y_val = validation_processed_numpy.next()

In [52]:
#convert the images to cropped faces
face_classifier = YOLO("../../app/src/face_detection/model/yolov8n-face.pt")

X_cropped_train = []
y_train_cropped = []

n_train = X_train.shape[0]
for i in range(n_train):
    face = X_train[i]
   
    faces = face_classifier.predict(face)
    boxes = faces[0].boxes.xyxy.tolist()
    if boxes:
        
        #extract the face based on the output from the YOLOv8 model        
        left, bottom, right, top = boxes[0]
        cropped_face = face[int(bottom):int(top), int(left):int(right)]
        #resize cropped face to a std shape, 100x100 for now but can adjust this
        pil_face = Image.fromarray(np.uint8(cropped_face))
        pil_face = pil_face.resize((48, 48))
        numpy_cropped_face = np.array(pil_face)
        #append this to the new list containing all cropped faces
        X_cropped_train.append(numpy_cropped_face)
        y_train_cropped.append(y_train[i])

X_val_cropped = []
y_val_cropped = []
n_val = X_val.shape[0]

for i in range(n_val):
    face = X_val[i]
    faces = face_classifier.predict(face)
    boxes = faces[0].boxes.xyxy.tolist()
    if boxes:
        
        #extract the face based on the output from the YOLOv8 model        
        left, bottom, right, top = boxes[0]
        cropped_face = face[int(bottom):int(top), int(left):int(right)]
        #resize cropped face to a std shape, 100x100 for now but can adjust this
        pil_face = Image.fromarray(np.uint8(cropped_face))
        pil_face = pil_face.resize((48, 48))
        numpy_cropped_face = np.array(pil_face)
        #append this to the new list containing all cropped faces
        X_val_cropped.append(numpy_cropped_face)
        y_val_cropped.append(y_val[i])



0: 640x640 1 face, 122.1ms
Speed: 1.1ms preprocess, 122.1ms inference, 0.8ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 97.7ms
Speed: 8.5ms preprocess, 97.7ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 86.2ms
Speed: 15.0ms preprocess, 86.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 82.8ms
Speed: 16.0ms preprocess, 82.8ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 96.8ms
Speed: 17.0ms preprocess, 96.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 87.4ms
Speed: 0.0ms preprocess, 87.4ms inference, 12.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 2 faces, 97.0ms
Speed: 0.0ms preprocess, 97.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 face, 71.4ms
Speed: 6.0ms preprocess, 71.4ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)



In [53]:
#change the variables to numpy arrays
X_cropped_train = np.array(X_cropped_train)

X_val_cropped = np.array(X_val_cropped)

print(X_cropped_train.shape)
print(X_val_cropped.shape)

#Flatten the RGB image into a flat tensor
X_cropped_train = X_cropped_train.reshape((X_cropped_train.shape[0], X_cropped_train.shape[1] * X_cropped_train.shape[2] * X_cropped_train.shape[3]))
X_val_cropped = X_val_cropped.reshape((X_val_cropped.shape[0], X_val_cropped.shape[1] * X_val_cropped.shape[2] * X_val_cropped.shape[3]))

(113, 48, 48, 3)
(26, 48, 48, 3)


In [54]:
y_train_lab = np.array(y_train_cropped).argmax(axis = 1)
y_val_lab = np.array(y_val_cropped).argmax(axis = 1)

In [55]:
pca = PCA(n_components = 50)

X_train_lower_dim = pca.fit_transform(X_cropped_train)

In [56]:
svc_cropped = SVC()

svc_cropped.fit(X_cropped_train, y_train_lab)

SVC()

In [57]:
y_pred = svc_cropped.predict(X_val_cropped)
#display the accuracy of the model
print(accuracy_score(y_val_lab, y_pred))
#note that the accuracy is lower compared to not using the cropped image


0.4230769230769231
